## Challenge

Las preguntas son las siguientes: 
1.	¿Qué cervecería produce la cerveza más fuerte según ABV? 
2.	¿Si tuviera que elegir 3 cervezas para recomendar usando sólo estos datos, cuáles elegiría? 
3.	¿Cuál de los factores (aroma, taste, apperance, palette) es más importante para determinar la calidad general de una cerveza? 
4.	¿Si yo típicamente disfruto una cerveza debido a su aroma y apariencia, qué estilo de cerveza debería probar? 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Inspeccionando el dataset

In [2]:
!ls

LICENSE          README.md        beer_reviews.csv challenge.ipynb


In [3]:
df = pd.read_csv("beer_reviews.csv")

In [4]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [5]:
print("Cantidad de datos:",df.shape[0])
print("Cantidad de columnas:",df.shape[1])

Cantidad de datos: 1586614
Cantidad de columnas: 13


In [6]:
# dtypes of attributes
df.dtypes

brewery_id              int64
brewery_name           object
review_time             int64
review_overall        float64
review_aroma          float64
review_appearance     float64
review_profilename     object
beer_style             object
review_palate         float64
review_taste          float64
beer_name              object
beer_abv              float64
beer_beerid             int64
dtype: object

Notamos que hay cervezas que se encuentran repetidas, es decir, tienen más de un registro hecho por distintos _reviewers_ y en distintos tiempos.

In [36]:
# cantidad de tipos de cervezas
df["beer_name"].value_counts()

90 Minute IPA                                   3290
India Pale Ale                                  3130
Old Rasputin Russian Imperial Stout             3111
Sierra Nevada Celebration Ale                   3000
Two Hearted Ale                                 2728
Arrogant Bastard Ale                            2704
Stone Ruination IPA                             2704
Sierra Nevada Pale Ale                          2587
Stone IPA (India Pale Ale)                      2575
Pliny The Elder                                 2527
Founders Breakfast Stout                        2502
Pale Ale                                        2500
Sierra Nevada Bigfoot Barleywine Style Ale      2492
La Fin Du Monde                                 2483
60 Minute IPA                                   2475
Storm King Stout                                2452
Duvel                                           2450
Brooklyn Black Chocolate Stout                  2447
Bell's Hopslam Ale                            

### Pregunta 1

La opción más obvia sería ordenar el `dataframe` por la columna `beer_abv`. Sin embargo dado la gran cantidad de datos sería muy ineficiente. En cambio se iterará sobre el `dataframe` guardando datos de las $10$ cervezas más fuertes: 

In [39]:
df1 = df[["beer_abv", "brewery_name", "beer_style", "beer_name", "beer_beerid"]]
df1.head()

,beer_abv,brewery_name,beer_style,beer_name,beer_beerid
0,5.0,Vecchio Birraio,Hefeweizen,Sausa Weizen,47986
1,6.2,Vecchio Birraio,English Strong Ale,Red Moon,48213
2,6.5,Vecchio Birraio,Foreign / Export Stout,Black Horse Black Beer,48215
3,5.0,Vecchio Birraio,German Pilsener,Sausa Pils,47969
4,7.7,Caldera Brewing Company,American Double / Imperial IPA,Cauldron DIPA,64883


In [40]:
n = 10 # n cervezas mas fuertes
max_abv_data = [(-float("inf"),None,None,None,None)]*n
beer_ids = []

for index,row in df1.iterrows():
    if row["beer_abv"] > max_abv_data[-1][0]:
        if row["beer_beerid"] in [x[-1] for x in max_abv_data]:
            continue # no contar cervezas repetidas
        max_abv_data.append( (row["beer_abv"], row["brewery_name"], row["beer_style"], row["beer_name"], row["beer_beerid"]))
        max_abv_data.sort(reverse=True)
        max_abv_data = max_abv_data[:n]

In [42]:
for br_abv,bw_name,br_style,br_name,_ in max_abv_data:
    print ("{0} - {1} - {2} - {3}".format(bw_name,br_name,br_style,br_abv))

Schorschbräu - Schorschbräu Schorschbock 57% - Eisbock - 57.7
Schorschbräu - Schorschbräu Schorschbock 43% - Eisbock - 43.0
BrewDog - Sink The Bismarck! - American Double / Imperial IPA - 41.0
Schorschbräu - Schorschbräu Schorschbock 40% - Eisbock - 39.44
De Struise Brouwers - Black Damnation VI - Messy - American Double / Imperial Stout - 39.0
BrewDog - Tactical Nuclear Penguin - American Double / Imperial Stout - 32.0
Schorschbräu - Schorschbräu Schorschbock 31% - Eisbock - 30.86
Hair of the Dog Brewing Company / Brewery and Tasting Room - Dave - English Barleywine - 29.0
BrewDog - Ghost Deer - Belgian Strong Pale Ale - 28.0
Boston Beer Company (Samuel Adams) - Samuel Adams Utopias - American Strong Ale - 27.0


De los resultados se ve que la cervecería _Schorschbräu_ produce las dos cervezas más fuertes. Por lo tanto se infiere que es está cervecería la que produce la cerveza más fuerte según ABV en el dataset.

### Pregunta 2

En este pregunta el atributo de mayor importancia es `review_overall`. Sin embargo también es importante fijarse en el tiempo en el que se hizo el review, por ello se considerará solo el review más actual, pues puede que una misma cerveza haya sido muy buena en el pasado pero lo haya dejado de ser en el presente. 

También se tomará en cuenta el estilo de la cerveza; Si se van a recomendar sólo 3 cervezas, sería bueno que no fueran todas del mismo tipo.

In [32]:
# cantidad de tipos de cervezas
df["beer_style"].value_counts()

American IPA                           117586
American Double / Imperial IPA          85977
American Pale Ale (APA)                 63469
Russian Imperial Stout                  54129
American Double / Imperial Stout        50705
American Porter                         50477
American Amber / Red Ale                45751
Belgian Strong Dark Ale                 37743
Fruit / Vegetable Beer                  33861
American Strong Ale                     31945
Belgian Strong Pale Ale                 31490
Saison / Farmhouse Ale                  31480
American Adjunct Lager                  30749
Tripel                                  30328
Witbier                                 30140
Hefeweizen                              27908
American Barleywine                     26728
American Brown Ale                      25297
American Stout                          24538
American Pale Wheat Ale                 24204
Märzen / Oktoberfest                    23523
English Pale Ale                  

Sacamos y guardamos los datos de interés en el diccionario `beer_map`, dejando los datos del review más reciente para cada cerveza.

In [60]:
beer_map = dict()

#for index,row in df.iterrows():
#    if row["beer_name"] not in beer_map:
#        beer_map[row["beer_name"]] = (row["review_overall"], row["review_time"], row["beer_style"])
#    elif row["review_time"] > beer_map[row["beer_name"]][1]:
#        # en caso de que ya este, verificamos el tiempo de review
#        beer_map[row["beer_name"]] = (row["review_overall"], row["review_time"], row["beer_style"])

for index,row in df.iterrows():
    if row["beer_name"] not in beer_map:
        beer_map[(row["beer_name"],row["beer_style"])] = [row["review_overall"]]
    else:
        beer_map[(row["beer_name"],row["beer_style"])].append( row["review_overall"] )

for key,ll in beer_map.items():
    beer_map[key] = sum(ll)/len(ll)

Se busca en el diccionario anterior las $N$ cervezas con mayor `review_overall`

In [64]:
n = 100
top_beers = [ (-float("inf"),None,None) ]*n

for key,overall in beer_map.items():
    name,style = key
    if overall > top_beers[-1][0]:
        top_beers.append( (overall,name,style) )
        top_beers.sort(reverse=True)
        top_beers = top_beers[:n]

In [65]:
top_beers

[(5.0, 'Zamboni Black Ice', 'Dunkelweizen'),
 (5.0, 'Woodstock Wheat', 'Hefeweizen'),
 (5.0, 'Wobbly Bob APA', 'American Pale Ale (APA)'),
 (5.0, 'Winter Ale', 'Winter Warmer'),
 (5.0, 'Wheat', 'Hefeweizen'),
 (5.0, 'Viven Porter', 'American Porter'),
 (5.0, 'Triple OOO', 'English Bitter'),
 (5.0, 'Triple Chocolate Stout', 'American Stout'),
 (5.0, 'Trafalgar Abbey Belgian Spice Ale', 'Belgian Dark Ale'),
 (5.0, 'The Big O', 'American Blonde Ale'),
 (5.0,
  'Tall Tale Pale Ale (dry-hopped With Citra)',
  'American Pale Ale (APA)'),
 (5.0, 'Summer Wheat', 'Hefeweizen'),
 (5.0, 'Suicide By Hops', 'American IPA'),
 (5.0, 'Sub Zero Lager', 'American Pale Lager'),
 (5.0, 'Steambock', 'Maibock / Helles Bock'),
 (5.0, 'Spanish Springs Porter', 'American Porter'),
 (5.0, 'Southern Belle', 'American Pale Ale (APA)'),
 (5.0, 'Schwindel Alt', 'Altbier'),
 (5.0, 'Saisonnaire', 'Saison / Farmhouse Ale'),
 (5.0, 'Rye Saison', 'Saison / Farmhouse Ale'),
 (5.0, "Rose City Till I Die'PA", 'American Dou

In [58]:
n = 500
top_beers = [ (-float("inf"),None,None) ]*n

for name,v in beer_map.items():
    overall,_,style = v
    if overall > top_beers[-1][0]:
        top_beers.append( (overall,name,style) )
        top_beers.sort(reverse=True)
        top_beers = top_beers[:n]

In [59]:
top_beers

[(5.0, 'Zwiesel Löwentrunk', 'German Pilsener'),
 (5.0, 'Zhu Jiang Draft Beer', 'Euro Pale Lager'),
 (5.0, 'Zamboni Black Ice', 'Dunkelweizen'),
 (5.0, 'Yorkshire Porter', 'American Porter'),
 (5.0, 'Yarmouth Town Brown', 'English Brown Ale'),
 (5.0, 'X', 'Baltic Porter'),
 (5.0, 'Wobbly Bob APA', 'American Pale Ale (APA)'),
 (5.0, 'Winter Wonderland', 'Herbed / Spiced Beer'),
 (5.0, 'Winter Wheat Bock', 'Weizenbock'),
 (5.0, 'Willamette Pale Ale', 'American Pale Ale (APA)'),
 (5.0, 'Wild Dog Coffee Stout', 'American Stout'),
 (5.0, 'White Zombie Ale', 'Witbier'),
 (5.0, 'White Winter Belgian Ale', 'Witbier'),
 (5.0, 'Whiskey Barrel Imperial Stout', 'Russian Imperial Stout'),
 (5.0, 'Wheelie', 'American Pale Ale (APA)'),
 (5.0, 'Wexford Wheat', 'American Pale Wheat Ale'),
 (5.0, 'Wet Hop Citra Ale', 'American Pale Ale (APA)'),
 (5.0, 'Weetwood Best Cask Bitter', 'English Bitter'),
 (5.0, 'Wasabi Brew', 'Herbed / Spiced Beer'),
 (5.0, 'Warrior Strong Pale Ale', 'American Pale Ale (APA)'

### Pregunta 3

In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

La estrategia a seguir será la siguiente: Se supondrá que existe una relación lineal entre las $4$ variables de interés y el `review_overall`. Se entrenarán $4$ regresores lineales (en un respectivo _training set_), cada uno omitiendo una de las variables. 

Se calculará el _score_ (coeficiente de determinación) de cada regresor obtenido (sobre un _testing set_). Cuanto más bajo sea el _score_, esto indicará que la variable que estamos omitiendo es importante!. Bajo dicho criterio se determinará la importancia de cada variable.


In [81]:
y = df["review_overall"].as_matrix()
X = df.as_matrix(columns=["review_aroma", "review_appearance", "review_palate", "review_taste"])

# Se divide training y testing en 75% y 25% respectivamente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [87]:
lr1 = LinearRegression()
# excluding review_taste
lr1.fit(X_train[:,[0,1,2]],y_train)
print("Coefficient of determination:", lr1.score(X_test[:,[0,1,2]], y_test))


Coefficient of determination: 0.5444870660229477


In [88]:
lr2 = LinearRegression()
# excluding review_palatate
lr2.fit(X_train[:,[0,1,3]],y_train)
print("Coefficient of determination:", lr2.score(X_test[:,[0,1,3]], y_test))

Coefficient of determination: 0.6285849919483448


In [90]:
lr3 = LinearRegression()
# excluding review_appearance
lr3.fit(X_train[:,[0,2,3]],y_train)
print("Coefficient of determination:", lr3.score(X_test[:,[0,2,3]], y_test))

Coefficient of determination: 0.6531073161147438


In [91]:
lr4 = LinearRegression()
# excluding review_aroma
lr4.fit(X_train[:,[1,2,3]],y_train)
print("Coefficient of determination:", lr4.score(X_test[:,[1,2,3]], y_test))

Coefficient of determination: 0.6527652817980704


Los resultados indican que las variables más importantes para determinar la calidad `overall` de la cerveza es (de mayor a menor):
1. `review_taste`
2. `review_palatate`
3. `review_aroma`
4. `review_appearance`

### Pregunta 4

In [68]:
c = 0
for index,row in df.iterrows():
    if row["review_overall"]==5.0:
        print(row)
        print("\n")
        c +=1
        if c==3: break

brewery_id                               1075
brewery_name          Caldera Brewing Company
review_time                        1283154365
review_overall                              5
review_aroma                                5
review_appearance                           4
review_profilename               MadeInOregon
beer_style               Herbed / Spiced Beer
review_palate                               4
review_taste                                4
beer_name                 Caldera Ginger Beer
beer_abv                                  4.7
beer_beerid                             52159
Name: 11, dtype: object


brewery_id                               1075
brewery_name          Caldera Brewing Company
review_time                        1318802642
review_overall                              5
review_aroma                                5
review_appearance                         3.5
review_profilename                optimator13
beer_style                          Rauchbier
review_p